# Introduction

The notebook is intened to experiment with different technologies for the task of **Question and Answering**.

There are two different **type of models**:
- *Open Domain* - They do not require a passed context
- *Reading Comprehension* - They find the answer within a given context

Such models can work in two different **approaches**:
- *Open Book* - The model can access external source of information
- *Closed Book* - The model can only access what has been encoded in its paramters

The **Components** of an Open Domain Q&A are:
- *Retriever* - It finds relevant contexts from an external source given the question (This is the component that differentiate an Open Domain from a Reading Comprehension)
- *Reader* - It locates the position in the context where the answer to the question is (alternatively there can be a *Generator*)
- *Generator*



## Datasets

### SQuAD 2.0

The SQuAD (Stanford Question and Answering Dataset) is a hugely popular dataset containing question and answer pairs scraped from Wikipedia, covering topics ranging from Beyonce, to Physics. 